In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# abrir o arquivo CSV
df = pd.read_csv("vendas_modificado.csv")

# Ordenando por id_da_compra
df = df.sort_values(by="id_da_compra")

# Selecionando somente as primeiras 1000 linhas
df = df.head(1000)

# Exibindo somente as primeiras 20 linhas
display(df.head(20))

# Exibir  os dados
df.info()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
119876,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,"R$ 288,20000000000005",7,25.40,Aguardando Pgto,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Lux
143431,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,"R$ 2,4",15,46.00,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Nivea
4853,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Macarrão,"R$ 2,63",14,46.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,3642XXXX,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Renata
364791,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Pasta de Dente,"R$ 2,98",9,36.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Colgate
170364,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Sabonete,"R$ 2,37",9,21.33,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Dove
102491,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Detergente,"R$ 3,45",15,51.75,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Minuano
219754,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Macarrão,"R$ 3,23",8,25.84,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Barilla
158811,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Refrigerante,"R$ 6,83",5,34.15,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Fanta
40805,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Água Mineral,"R$ 1,4",10,23.00,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Minalba
86920,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Macarrão,"R$ 3,76",6,31.56,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Barilla


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 119876 to 345830
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_da_compra  1000 non-null   int64  
 1   data          1000 non-null   object 
 2   hora          1000 non-null   object 
 3   cliente       1000 non-null   object 
 4   produto       1000 non-null   object 
 5   valor         1000 non-null   object 
 6   quantidade    1000 non-null   int64  
 7   total         991 non-null    float64
 8   status        1000 non-null   object 
 9   cidade        1000 non-null   object 
 10  estado        1000 non-null   object 
 11  pais          1000 non-null   object 
 12  cep           1000 non-null   object 
 13  frete         984 non-null    float64
 14  pagamento     1000 non-null   object 
 15  vendedor      993 non-null    object 
 16  marca         1000 non-null   object 
dtypes: float64(2), int64(2), object(13)
memory usage: 140.6+ KB


In [ ]:
# A coluna "valor" apresenta casas decimais inconsistentes e estão no formato de string
# Irei mudar para float, remover R$ e manter somente 2 casas decimais após o "." deixando padronizado.

# Remove espaços extras antes e depois do valor
df['valor'] = df['valor'].str.strip()

# Remove "R$" e possiveis espaços
df['valor'] = df['valor'].str.replace(r'R\$', '', regex=True).str.strip()

# Convertendo para float e substituindo "," para "."
df['valor'] = df['valor'].str.replace(',', '.', regex=True).astype(float)

# Definindo para duas casas decimais
df['valor'] = df['valor'].round(2)

# Exibir para conferir
display(df.head(15))


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
119876,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,288.20,7,25.40,Aguardando Pgto,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Lux
143431,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,2.40,15,46.00,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Nivea
4853,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Macarrão,2.63,14,46.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,3642XXXX,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Renata
364791,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Pasta de Dente,2.98,9,36.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Colgate
170364,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Sabonete,2.37,9,21.33,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Dove
102491,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Detergente,3.45,15,51.75,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Minuano
219754,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Macarrão,3.23,8,25.84,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Barilla
158811,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Refrigerante,6.83,5,34.15,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Fanta
40805,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Água Mineral,1.40,10,23.00,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Minalba
86920,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Macarrão,3.76,6,31.56,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Barilla


In [13]:
# Confirma se as datas e horas estão no mesmo formato
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Deixando a hora padrão
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time

In [9]:
# Algumas colunas deveriam ter sempre o mesmo valor dentro da mesma compra (id_da_compra)

# Cria um DataFrame vazio para armanzenar diferenças
compras_com_diferencas = pd.DataFrame()

# Lista das colunas que devem ser iguais em todas as linhas da mesma compra
colunas_fixas = ["data", "hora", "cliente", "status", "cidade", "estado", "pais", "cep", "frete", "vendedor"]

# Para cada coluna dessa lista
for coluna in colunas_fixas:
    # Pegar o valor mais comum dessa coluna dentro de cada compra
    correcoes = df.groupby("id_da_compra")[coluna].agg(lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0])

    # Substituir os valores diferentes pelo valor mais comum da compra
    for id_compra, valor_correto in correcoes.items():
        df.loc[df["id_da_compra"] == id_compra, coluna] = valor_correto

display(df)


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
119876,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,288.20,7,25.40,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Lux
143431,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,2.40,15,46.00,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Nivea
4853,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Macarrão,2.63,14,46.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Renata
364791,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Pasta de Dente,2.98,9,36.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Colgate
170364,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Sabonete,2.37,9,21.33,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Dove
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249416,166,2019-02-08,17:16:04,MATEUS VITOR FAÇANHA NEVES,Presunto,10.05,5,60.75,Em Transporte,Congonhas,MG,Brasil,36415-000,10.5,Pix,HENRICO MATOS LIMA DA CUNHA,Perdigão
235575,166,2019-02-08,17:16:04,MATEUS VITOR FAÇANHA NEVES,Feijão,8.18,1,18.68,Em Transporte,Congonhas,MG,Brasil,36415-000,10.5,Pix,HENRICO MATOS LIMA DA CUNHA,Caldo Bom
350503,166,2019-02-08,17:16:04,MATEUS VITOR FAÇANHA NEVES,Açúcar,3.46,1,13.96,Em Transporte,Congonhas,MG,Brasil,36415-000,10.5,Pix,HENRICO MATOS LIMA DA CUNHA,Caravelas
13164,166,2019-02-08,17:16:04,MATEUS VITOR FAÇANHA NEVES,Shampoo,6.27,11,79.47,Em Transporte,Congonhas,MG,Brasil,36415-000,10.5,Pix,HENRICO MATOS LIMA DA CUNHA,Pantene


In [10]:
# Removendo caracteres especiais
df["produto"] = df["produto"].str.replace(r'[^a-zA-ZáéíóúãõçÁÉÍÓÚÃÕÇ\s]', '', regex=True)

display(df.head(20))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
119876,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,288.20,7,25.40,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Lux
143431,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,2.40,15,46.00,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Nivea
4853,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Macarrão,2.63,14,46.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Renata
364791,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Pasta de Dente,2.98,9,36.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Colgate
170364,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Sabonete,2.37,9,21.33,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Dove
102491,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Detergente,3.45,15,51.75,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Minuano
219754,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Macarrão,3.23,8,25.84,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Barilla
158811,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Refrigerante,6.83,5,34.15,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Fanta
40805,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Água Mineral,1.40,10,23.00,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Minalba
86920,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Macarrão,3.76,6,31.56,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Barilla


In [11]:
# Verifica se existe valores nulos
display(df.isnull().sum())


id_da_compra    0
data            0
hora            0
cliente         0
produto         0
valor           0
quantidade      0
total           9
status          0
cidade          0
estado          0
pais            0
cep             0
frete           0
pagamento       0
vendedor        0
marca           0
dtype: int64

In [12]:
# Foi identificado que na coluna "total" existe valores nulos
# multiplicando a coluna "valor" vezes "quantidade"
df['total'] = df['valor'] * df['quantidade'] + df['frete']

display(df.head(15)) 

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
119876,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,288.20,7,2027.40,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Lux
143431,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Sabonete,2.40,15,46.00,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Nivea
4853,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Macarrão,2.63,14,46.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Renata
364791,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,Pasta de Dente,2.98,9,36.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Colgate
170364,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Sabonete,2.37,9,21.33,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Dove
102491,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Detergente,3.45,15,51.75,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Minuano
219754,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Macarrão,3.23,8,25.84,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Barilla
158811,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,Refrigerante,6.83,5,34.15,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Fanta
40805,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Água Mineral,1.40,10,23.00,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Minalba
86920,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,Macarrão,3.76,6,31.56,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Barilla


In [14]:
# Seleciona colunas numéricas
colunas_numericas = df.select_dtypes(include='number').columns

# Listas para armazenar os resultados
colunas_com_negativos = []
colunas_sem_negativos = []

# Verificando as coluna
for col in colunas_numericas:
    if (df[col] < 0).any():
        colunas_com_negativos.append(col)
        print(f"Coluna '{col}' tem valores negativos:")
        print(df[df[col] < 0][[col]].head())
        print()
    else:
        colunas_sem_negativos.append(col)


df[colunas_numericas] = df[colunas_numericas].abs()
# Todos os valores que negativos foram transformados em positivos

In [ ]:
# Seleciona apenas colunas numéricas
colunas_numericas = df.select_dtypes(include='number').columns

# Listas para armazenar os resultados
colunas_com_negativos = []
colunas_sem_negativos = []

# Verifica cada coluna
for col in colunas_numericas:
    if (df[col] < 0).any():
        colunas_com_negativos.append(col)
        print(f"Coluna '{col}' tem valores negativos:")
        print(df[df[col] < 0][[col]].head())
        print()
    else:
        colunas_sem_negativos.append(col)


df[colunas_numericas] = df[colunas_numericas].abs()
#agora todos os valores que estavam negativos foram transformados em positivos

In [15]:
# Identificando e tratando outliers com base no valor de cada produto
def tratar_outliers_por_produto(df, col_produto='produto', col_valor='valor'):
    df = df.copy()  # Cria uma cópia do DataFrame para não alterar o original
    resumo_outliers = []  # Lista para armazenar um resumo dos outliers encontrados por produto

    # Agrupa os dados por produto
    for produto, grupo in df.groupby(col_produto):
        # Calcula IQR
        q1 = grupo[col_valor].quantile(0.25)
        q3 = grupo[col_valor].quantile(0.75)
        iqr = q3 - q1

        # Define os limites inferior e superior para considerar um valor como outlier
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        # Cria uma condição para localizar os outliers de um produto específico
        condicao_outlier = (df[col_produto] == produto) & (
            (df[col_valor] < limite_inferior) | (df[col_valor] > limite_superior)
        )

        # Separa os outliers e calcula estatísticas
        outliers = df[condicao_outlier]
        total_produto = len(grupo)
        total_outliers = len(outliers)
        perc = (total_outliers / total_produto) * 100 if total_produto else 0

        # Exibe informações se houver outliers no produto
        if total_outliers > 0:
            print(f"\nProduto: {produto}")
            print(f"Limites: {limite_inferior:.2f} a {limite_superior:.2f}")
            print(f"Outliers encontrados: {total_outliers} de {total_produto} ({perc:.2f}%)")
            print(outliers[[col_produto, col_valor]])

        # Adiciona os dados ao resumo
        resumo_outliers.append({
            'produto': produto,
            'total': total_produto,
            'outliers': total_outliers,
            'percentual': round(perc, 2)
        })

        # Substitui os outliers por valores nulos (NaN)
        df.loc[condicao_outlier, col_valor] = None

    # Cria e exibe um DataFrame com o resumo dos outliers
    resumo_df = pd.DataFrame(resumo_outliers).sort_values(by='percentual', ascending=False)
    
    print("\nResumo dos outliers por produto:")
    print(resumo_df)

    return df  # Retorna o DataFrame com os outliers removidos (como NaN)


# Função para preencher valores nulos com a média do valor de cada produto
def preencher_valores_nulos_com_media(df, col_produto='produto', col_valor='valor'):
    df = df.copy()  # Cria uma cópia do DataFrame para preservar o original

    # Preenche os NaNs com a média de valor por produto
    df[col_valor] = df.groupby(col_produto)[col_valor].transform(
        lambda x: x.fillna(x.mean())
    )

    return df  # Retorna o DataFrame com os valores preenchidos


# 1. Substitui os outliers por NaN
df = tratar_outliers_por_produto(df)

# 2. Preenche os NaNs com a média de valor de cada produto
df = preencher_valores_nulos_com_media(df)



Produto: Açúcar
Limites: 1.70 a 4.25
Outliers encontrados: 1 de 56 (1.79%)
       produto   valor
196440  Açúcar  413.96

Produto: Biscoito Recheado
Limites: 2.17 a 3.74
Outliers encontrados: 1 de 29 (3.45%)
                  produto  valor
253620  Biscoito Recheado   3.77

Produto: Café
Limites: 5.17 a 14.81
Outliers encontrados: 1 de 50 (2.00%)
       produto    valor
368388    Café  1361.09

Produto: Condicionador
Limites: 4.19 a 16.70
Outliers encontrados: 2 de 24 (8.33%)
              produto    valor
185863  Condicionador   830.54
206628  Condicionador  1160.66

Produto: Desinfetante
Limites: 1.96 a 7.68
Outliers encontrados: 2 de 55 (3.64%)
             produto   valor
83243   Desinfetante  882.94
306947  Desinfetante  596.05

Produto: Farinha de Trigo
Limites: 1.52 a 4.57
Outliers encontrados: 1 de 20 (5.00%)
                 produto   valor
144442  Farinha de Trigo  358.94

Produto: Feijão
Limites: 6.62 a 10.10
Outliers encontrados: 3 de 26 (11.54%)
       produto  valor
2081

In [16]:
# Descobrindo associações entre produtos
# Agrupando os produtos por id da compra
transacoes = df.groupby('id_da_compra')['produto'].apply(list).tolist()

# Transformando formato binário para o Apriori
te = TransactionEncoder()
te_array = te.fit(transacoes).transform(transacoes)
df_binario = pd.DataFrame(te_array, columns=te.columns_)

# Aplicando Apriori
frequent_itemsets = apriori(df_binario, min_support=0.02, use_colnames=True)

# Gerando as regras de associação
regras = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

# Mostrando as regras
print(regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

            antecedents                                        consequents  \
0           (Amaciante)                                           (Açúcar)   
1              (Açúcar)                                        (Amaciante)   
2      (Pasta de Dente)                                        (Amaciante)   
3           (Amaciante)                                   (Pasta de Dente)   
4           (Amaciante)                                         (Sabonete)   
..                  ...                                                ...   
933   (Molho de Tomate)           (Café, Feijão, Queijo Mussarela, Açúcar)   
934            (Açúcar)  (Café, Molho de Tomate, Feijão, Queijo Mussarela)   
935            (Feijão)  (Café, Molho de Tomate, Queijo Mussarela, Açúcar)   
936  (Queijo Mussarela)            (Molho de Tomate, Feijão, Café, Açúcar)   
937              (Café)  (Molho de Tomate, Feijão, Queijo Mussarela, Aç...   

      support  confidence      lift  
0    0.023952    0.333333

Relatorio Final :

Durante o processo de análise e limpeza dos dados, iniciei carregando o arquivo vendas_modificado.csv e selecionei as primeiras 1000 linhas para facilitar a manipulação inicial. Ao examinar os dados com df.info(), identifiquei alguns problemas importantes que precisariam ser tratados antes de qualquer análise mais aprofundada.

A coluna "valor" apresentava problemas do tipo: os valores estavam no formato de string, acompanhados do símbolo "R$", com espaços desnecessários e utilizando vírgula como separador decimal. Fiz a limpeza removendo o "R$", substituí a vírgula por ponto, converti a coluna para o tipo float e padronizei todos os valores para duas casas decimais.

A coluna "total" continha valores nulos. Para corrigir isso, apliquei a fórmula de cálculo do total por item, que é o valor unitário multiplicado pela quantidade, somando o valor do frete: total = valor * quantidade + frete.

Em seguida, converti a coluna "data" para o formato datetime e a coluna "hora" para o formato HH:MM:SS, garantindo uniformidade para futuras análises temporais. Outro ponto importante foi a padronização de colunas que não deveriam variar dentro do mesmo id_da_compra, como data, hora, cliente, status, cidade, estado, país, CEP, frete e vendedor. Para isso, agrupei os dados por ID de compra e utilizei a moda para garantir que todos os registros de uma mesma compra tivessem as mesmas informações nessas colunas.

Também identifiquei valores negativos em colunas numéricas como "valor", "quantidade", "frete" e "total", o que não faz sentido no contexto da base. Corrigi isso transformando todos esses valores negativos em seus equivalentes positivos com a função .abs().

Para melhorar ainda mais a qualidade dos dados, realizei o tratamento de outliers na coluna "valor", analisando produto por produto. Apliquei a técnica do intervalo interquartil (IQR) para encontrar valores muito fora da curva. Os outliers foram substituídos por NaN e, posteriormente, esses valores nulos foram preenchidos com a média de valor de cada produto, mantendo a coerência dos dados sem perder registros.

Além disso, limpei a coluna "produto", removendo todos os caracteres especiais e mantendo apenas letras e espaços, com o objetivo de padronizar os nomes dos produtos para a etapa de análise.

Com a base devidamente limpa, organizada e padronizada, parti para a etapa de descoberta de associações entre produtos. Agrupei os produtos comprados em cada id_da_compra e utilizei o TransactionEncoder para transformar os dados em um formato binário, necessário para aplicar o algoritmo Apriori. A partir disso, identifiquei combinações frequentes de produtos comprados juntos e gerei regras de associação, com base em métricas como suporte, confiança e lift. Os resultados obtidos permitiram identificar padrões interessantes de consumo, que podem ser utilizados para estratégias de marketing ou vendas combinadas.

Esse foi o processo completo de preparação, limpeza e análise da base de dados de vendas. Todas as etapas foram fundamentais para garantir que os dados estivessem consistentes, confiáveis e prontos para gerar insights relevantes.